In [1]:
import pandas as pd
import numpy as np

In [2]:
from srcPy import AutoFAVAR, FAVAR

## 0. Read data

In [3]:
x, y = pd.read_csv('example_data/X.csv',index_col=0), pd.read_csv('example_data/Y.csv',index_col=0)
x.shape, y.shape

((100, 100), (100, 100))

In [4]:
y.head(3)

,Y1,Y2,Y3,Y4,Y5,Y6,Y7,Y8,Y9,Y10,...,Y91,Y92,Y93,Y94,Y95,Y96,Y97,Y98,Y99,Y100
time_idx,,,,,,,,,,,,,,,,,,,,,
1,0.614396,0.964794,0.205893,-0.648300,0.113615,-0.079261,0.381705,-0.532581,-0.443132,-0.312741,...,-0.439978,1.059277,-0.323260,-0.553377,0.235119,-1.101950,0.150781,-0.093707,0.384868,0.137204
2,0.043483,-0.311459,0.113461,0.109752,-0.356715,-0.581808,0.103916,0.140395,0.166327,-0.854355,...,-0.126835,-1.013673,-0.119078,0.267141,-0.602526,-0.366413,0.007523,-1.433163,0.308292,0.036191
3,0.268806,-0.465801,-0.022908,0.048832,0.830169,-0.796044,-0.150087,-0.000410,0.246142,-0.046778,...,0.236769,0.507834,-0.701815,-2.011438,0.622552,0.687118,-0.732307,0.252882,-0.377549,-0.210850


In [5]:
x.head(3)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X91,X92,X93,X94,X95,X96,X97,X98,X99,X100
time_idx,,,,,,,,,,,,,,,,,,,,,
1,-0.100736,-0.215206,-0.194921,0.086109,-0.141367,0.144609,-0.070564,0.074842,-0.006033,0.106137,...,-0.171001,-0.233844,0.072867,-0.111692,-0.154498,0.009950,0.065247,-0.015717,0.015995,0.093946
2,0.064581,0.132862,-0.263804,0.179877,-0.065660,0.399248,0.001430,-0.172531,0.000561,0.011380,...,-0.175317,0.113113,-0.006083,-0.256066,0.220495,-0.234262,-0.045540,-0.023510,-0.106902,-0.310751
3,-0.214974,-0.319049,0.145227,0.156510,-0.082962,-0.083639,-0.037105,-0.465667,-0.038461,0.084311,...,-0.061821,-0.034856,0.139800,-0.153172,0.088622,0.189861,-0.074970,0.091326,-0.201813,-0.014935


In [6]:
ydata, xdata = y.values, x.values ## get the numpy arrays

## 1. Set up tuning parameter range and run

In [7]:
rk_seq = np.arange(3,8,1)
lambdaB_seq = np.arange(0.25,3.25,0.25) * np.sqrt(np.log(xdata.shape[1])/xdata.shape[0])    # constant_range * sqrt(log(p)/n) which is the theoretical rate 
lambdaA_seq = None # default sequence will be used; alternatively can pass in a sequence per your liking

### use AutoFAVAR, where the code picks the optimal by doing grid search

In [8]:
model_auto = AutoFAVAR(IR = 'IC3',
                       max_iter = 500,
                       tol = 1.0e-4)

In [9]:
output = model_auto.autofit(ydata, xdata, 2, rk_seq, lambdaB_seq, lambdaA_seq, verbose=False)

[2024-02-08 22:20:44] Stage I: running autofit for the Information Eqn over a lattice
[2024-02-08 22:22:46] Done; r_best=3 (index=0), lambdaB_best=0.1073 (index=1); refitting ...
[2024-02-08 22:22:46] Start Information Equation Estimation
>>> Converged @ iter = 15; f_terminal = 24.7386
[2024-02-08 22:22:48] Stage II: running autofit for the VAR Eqn over a grid
[2024-02-08 22:22:49] Done; lambdaA_best=0.0217; refitting ...
[2024-02-08 22:22:49] Start VAR estimation
>>> Done; A.shape=(103, 103, 2)


In [10]:
output.keys()

dict_keys(['estA', 'estTheta', 'estF', 'estLambda', 'estB', 'pic', 'bic'])

### directly supply the values of the tuning parameters

In [11]:
model = FAVAR(IR = 'IC3', max_iter = 500, tol = 1.0e-4)

In [13]:
output = model.fit(ydata, xdata, 2, rk=5, lambdaB=0.1073, lambdaA=0.0217, verbose=2)

[2024-02-08 23:07:12] Start Information Equation Estimation
>> f_initial = 16.3420
>> iter=0, fB_delta=16.6726, fTheta_delta=-5.3957, f_delta=11.2769
>> iter=1, fB_delta=-5.1024, fTheta_delta=-0.0608, f_delta=-5.1632
>> iter=2, fB_delta=-0.4376, fTheta_delta=-0.0026, f_delta=-0.4403
>> iter=3, fB_delta=-0.1075, fTheta_delta=-0.0003, f_delta=-0.1078
>> iter=4, fB_delta=-0.0344, fTheta_delta=-0.0000, f_delta=-0.0344
>> iter=5, fB_delta=-0.0122, fTheta_delta=-0.0000, f_delta=-0.0122
>> iter=6, fB_delta=-0.0045, fTheta_delta=-0.0000, f_delta=-0.0045
>> iter=7, fB_delta=-0.0018, fTheta_delta=-0.0000, f_delta=-0.0018
>> iter=8, fB_delta=-0.0007, fTheta_delta=-0.0000, f_delta=-0.0007
>> iter=9, fB_delta=-0.0003, fTheta_delta=-0.0000, f_delta=-0.0003
>> iter=10, fB_delta=-0.0001, fTheta_delta=-0.0000, f_delta=-0.0001
>> iter=11, fB_delta=-0.0000, fTheta_delta=-0.0000, f_delta=-0.0000
>>> Converged @ iter = 12; f_terminal = 21.8536
[2024-02-08 23:07:15] Start VAR estimation
>>> Done; A.shape=(1